## What are we going to do?
This Notebook will guide you through the analysis of near-infrared (NIR) high-resolution spectroscopy (HRS) data of exoplanet atmospheres via cross-correlation techniques. In the literature this is sometimes called High-Resolution Cross-Correlation Spectroscopy (HRCCS). Compared to the first part that Jens just explained, we are going to look into the **infrared** and we are also looking at **emission spectroscopy** rather than transmission.

## The Earth's atmosphere in the NIR

The Earth's transmission spectrum longward of 1 $\mu$m becomes severely affected by the absorption of molecular species in our own atmosphere, primarily water vapour (H<sub>2</sub>O), but also methane (CH<sub>4</sub>) and carbon dioxide (CO<sub>2</sub>). An example of this can be seen in a model transmission spectrum calculated with the [ESO Sky Model Calculator](https://www.eso.org/observing/etc/bin/gen/form?INS.MODE=swspectr+INS.NAME=SKYCALC) (R=20,000, zenith, 2.5 mm of precipitable water vapour, average yearly conditions at Cerro Paranal).
![Earth's transsmission spectrum](figs/transmission.gif)
Such dominance of telluric lines (telluric = "of the Earth") requires a different approach than for optical spectra. 

## Getting started - Emission spectroscopy of exoplanet atmospheres

To add another layer of complexity, we are not going to work with transmission spectroscopy but with *emission* spectroscopy. Thus, rather than having the planet crossing in front of the stellar disk, it will be observed approximately half-orbit later, when the dayside of the planet is in full view.
![Exoplanet Spectroscopy](figs/atm_trans.png)
**NOTE:** At low spectral resolution (i.e. HST/WFC3) and in photometry (i.e. Spitzer IRAC), you are used to think to the planet's emission measured as the *drop in flux* during secondary eclipse. For reasons that will soon become clear, HR spectroscopy normalises each spectrum relative to the stellar spectrum, hence **secondary eclipses cannot be measured**. We instead observe before or after secondary eclipse, i.e. when both the stellar and the planet spectrum are present. 

**RECALL:** What allows us to separate the two is the changing Doppler shift of the spectrum. This is explained by the figure below, where the telluric lines (stationary in wavelengths) are in black and the planet's lines (Doppler shifting) in white. Since we do not need the secondary eclipse anymore, **HR emission spectroscopy can also be done on non-transiting planets**.
![Toy model of exoplanet spectroscopy at high spectral resolution](figs/toy.png)

## Step 1 - Loading in the data

We are going to work with emission spectroscopy of a non-transiting hot Jupiter, HD 179949 b, originally published in [(Brogi et al. 2014)](https://ui.adsabs.harvard.edu/abs/2014A%26A...565A.124B/abstract). This is a [bright](http://simbad.u-strasbg.fr/simbad/sim-id?Ident=hd+179949&NbIdent=1&Radius=2&Radius.unit=arcmin&submit=submit+id) late-F star in the Southern hemisphere well visible from Cerro Paranal.

These data were spread across three nights of observations with CRIRES at the Very Large Telescope, for a total of about 14 hours of observations. The relevant vectors for each of the night are saved in the sub-folder <code>nightX/</code>, where <code>X = [1,2,3]</code>.

Let us start by looking at the data from the first night, but we will then repeat the process on the other two nights as well.

In [ ]:
# Only basic packages necessary
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt

dataFold = 'night1/'

# It might look like an overkill to write a function to do this, but it will be handy later, 
# when we loop through three nights
def get_night_vectors(dataFold):
    flux, = np.load(dataFold+'flux.npy') # Recorded fluxes - Skipping detector 4
    wlen, = np.load(dataFold+'wlen.npy') # Pixel-wavelength solution - Skipping detector 4
    air, = np.load(dataFold+'air.npy')   # Airmass of the observations (1 = zenith, 2 ~ 30 deg above horizon)
    rvel, = np.load(dataFold+'rvel.npy') # Systemic velocity - barycentric correction (see next cells)
    ph, = np.load(dataFold+'ph.npy')     # Planet's orbital phase computed as in Jens' notebook
    return flux[0:3,], wlen[0:3,], air, rvel, ph

flux, wlen, air, rvel, ph = get_night_vectors(dataFold)

In [ ]:
''' Plotting useful information about the night '''

plt.figure(figsize=(12,3))
plt.subplot(1,2,1)
plt.plot(ph,air,'+')
plt.xlabel('Orbital phase')
plt.ylabel('Airmass of observations')
plt.subplot(1,2,2)
plt.plot(ph,rvel,'+')
plt.xlabel('Orbital phase')
plt.ylabel("$V_\mathrm{sys} - V_\mathrm{bary}$ (km/s)")
plt.show()

The airmass of these observations varies smoothly as expected when tracking an object in the sky, and over a wide range, from zenith (the small gap is due to the telescope being unable to track close to zenith) to the VLT pointing limit of 23 degrees above horizon.

The orbital phase is just > 0.5. This means we are tracking the planet post-superior conjunction (secondary eclipse if it were transiting).

The vector <code>rvel</code> contains the sum of the **systemic velocity** *V*<sub>sys</sub> of the object **minus the barycentric correction** *V*<sub>bary</sub>. This minus sign is necessary because the barycentric correction is the velocity of the observer in the centre of mass of the solar system, whereas we want the opposite being the data calibrated in the observer's reference frame. 

**Useful tip: Always check the definition of your barycentric correction and make sure to apply the right sign.**

In [ ]:
''' Getting dimension of the data '''
nDet, nObs, nPix = flux.shape
print(flux.shape)

''' Again, this might look like an overkill but we will need to plot datacubes constantly,
so we might as well write a function for it. '''
def plotMatrix(mat,xvec,yvec,xlab='',ylab='',stretch=False):
    if stretch:
        vmin = np.mean(mat)-np.std(mat) * 3
        vmax = np.mean(mat)+np.std(mat) * 3
    else:
        vmin = mat.min()
        vmax = mat.max()
    ext = [xvec[0],xvec[-1],yvec[0],yvec[-1]]
    plt.figure(figsize=(12,3))
    plt.imshow(mat,origin='lower',aspect='auto',extent=ext,vmin=vmin,vmax=vmax)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.colorbar()
    plt.show()
    
for iDet in range(nDet):
    plotMatrix(flux[iDet,],wlen[iDet,],ph,'Wavelength (nm)','Orbital phase')

You should now be familiar with the way the data is organised from what Jens taught you. Here instead of *spectral orders* we've got *detectors* (CRIRES has four of them, but we're only using three). The time dimension is along each column (i.e. each row is an observed spectrum - 199 in total for the night), and wavelength is along each row (i.e. each column is a specific wavelength - with the pixel-wavelenth soluton specified by <code>wlen</code>, which has shape <code>(nDet, nPix)</code>.

The **unit of flux** here is ADU per second of integration. To derive the total number of electrons recorded, you would have to multiply by the exposure time per spectrum (constant = 45 s) and the gain of the detector (approx. 7 e-/ADU).

Compared to the HARPS spectra, you will notice a lot more vertical lines, which are *telluric* lines. This supports our initial considerations about NIR spectroscopy being dominated by tellurics. However, there is enough "room" in between telluric lines to allow spectroscopy at R=100,000 and also most of the telluric lines are unblended. If you knew where to look, you would also spot some weak *stellar* absorption lines, but these are negligible in the *K* band for an F star.

You will also notice a much stronger variability across the time sequence than in the Kelt-9 data. This is partly due to the large range in airmass, partly to imperfection in the pointing, AO, guiding, etc., meaning that the fraction of the incident flux entering the slit varies from spectrum to spectrum. **This makes it impossible to do spectro-photometry** (i.e. produce a light curve vs wavelength) **with HR spectra**, a very important caveat to keep in mind.

**Note on stability:** Each CRIRES detector has 1,024 pixels, thus the length of the <code>nPix</code> dimension is also 1,024. However, that CRIRES is not as stable as HARPS, and therefore each spectrum in the sequence has been **realigned using the telluric lines as reference**, i.e. by measuring their pixel position in each spectrum and correcting any sub-pixel shift along the sequence. For reasons of time the alignment is not covered here, the only relevant info is that each pixel along the x-axis has always the same wavelength during one night, and that this is in *vacuum* and in the *observer's reference frame*, which is assumed to be equal to the telluric reference frame.

## Step 2 - removing telluric lines

This is the key part of HRCCS and we'll write code step by step to explain the process. Firstly, we are going to normalise each spectrum to remove througput variations (the horizontal stripes).

In [ ]:
''' Continuum normalisation - LIVE code '''

def normalise_continuum(flux,plot=False):
    medFlux = np.median(flux,axis=2) # Very rough proxy for continuum flux
    for iDet in range(nDet):
        for iObs in range(nObs):
            flux[iDet,iObs,] /= np.median(medFlux[iDet,iObs])
        if plot: plotMatrix(flux[iDet,],wlen[iDet,],ph,'Wavelength (nm)','Orbital phase')
    return flux, medFlux

fFlat, medFlux = normalise_continuum(flux.copy(),plot=True)

The continuum of the spectra is now approximately normalised. Those of you with a good eye will now notice that while telluric lines always fall onto the same pixel (the same column), their depth vary during the observing sequence. To convince ourselves that this is true, let's plot a column of the continuum and one tracking a telluric line:

In [ ]:
''' Testing different behaviour of continuum vs telluric lines - pixels selected by eye '''

cPix = 245  # Pixel number of a continuum region
tPix = 202  # Pixel number of a strong telluric line

plt.plot(np.arange(180,300),fFlat[0,0,180:300],c='k')
plt.plot([tPix],fFlat[0,0,tPix],'*',c='tab:blue')
plt.plot([cPix],fFlat[0,0,cPix],'*',c='tab:orange')
plt.ylabel('Normalised flux')
plt.xlabel('Pixel number')
plt.title('Small section of normalised spectrum')
plt.show()

In [ ]:
''' Slicing and normalising the continuum and telluric fluxes '''
cFlux = fFlat[0,:,cPix].copy()
tFlux = fFlat[0,:,tPix].copy()

cFlux /= cFlux.max()
tFlux /= tFlux.max()

plt.plot(ph,tFlux,label='Telluric line')     # Telluric line at 2.29 micron
plt.plot(ph,cFlux,label='Continuum')         # Continuum flux redwards of telluric line
plt.xlabel('Orbital phase')
plt.ylabel('Normalised flux')
plt.title('Time evolution of flux in two regions of the spectrum')
plt.legend()
plt.show()

The main culprit for this behaviour is **airmass**. Observing through progressively thicker layers of the atmosphere (higher airmass) increases absorption because the total optical depth increases. In regions with negligible opacity (i.e. the continuum) this effect is barely noticeable, but in the core of telluric lines it produces 30-40% variations. **Just normalising by an average master spectrum won't do the trick**. Let me show you this.

In [ ]:
''' Normalising by a median spectrum does not correct telluric lines '''

mat = fFlat[0,].copy()   # Copy of 1st detector
avgSpec = np.median(mat,axis=0)  # Time-averaged spectrum
for iObs in range(nObs): mat[iObs,] /= avgSpec  # Divide each spectrum by the time avg

plotMatrix(mat,wlen[0,],ph)

Let me instead show you that the previous telluric line at pixel <code>cPix</code> of detector 1 correlates well with airmass:

In [ ]:
''' Testing the correlation between the depth of a telluric line and airmass '''
xvec = air
yvec = tFlux
plt.plot(xvec,yvec,'+')
# Computing polynomial fit via np.polyfit() and np.poly1d()
fit = np.poly1d(np.polyfit(xvec,yvec,2))(xvec)

plt.plot(xvec,fit)
plt.show()

As you can see above, there is a tight correlation between the depth of a telluric line and airmass, but this is somehow more complex than just a linear correlation. Those of you with experience in radiative transfer might object that the depth of a line should correlate with the *exponential* of the airmass, and this is a slightly better approximation. However, for reasons related to the instability of the Earth's atmosphere on timescales of hours and the presence of additional instrumental effects, usually the correlation is at least quadratic even with the exp(airmass).

Given the evident correlation with airmass, we are going to code the simplest of the telluric removal algorithms designed for HRCCS, which is **airmass detrending**. We'll remove a quadratic fit between airmass and flux from each column (each wavelength bin) in the data.

In [ ]:
''' LIVE CODE - airmass de-trending (can be faster) '''

def detrend_tellurics(xvec, cube, deg=2, plot=False):
    nDet, nObs, nPix = cube.shape
    tRem = cube.copy()
    for iDet in range(nDet):
        for iPix in range(nPix):
            fit = np.poly1d(np.polyfit(xvec,cube[iDet,:,iPix],deg))(xvec)
            tRem[iDet,:,iPix] = cube[iDet,:,iPix] / fit
        if plot: plotMatrix(tRem[iDet,],wlen[iDet,],ph,'Wavelength (nm)','Orbital phase',stretch=True)
    return tRem

tRem = detrend_tellurics(air,fFlat,deg=2,plot=True)

The telluric correction just applied is far from perfect. While we have very substantially reduced the amplitude of telluric variations from 40% to just 1.5%, there are still very noticeable patterns (the vertical lines). These are due to changes in the abudance of telluric species (mainly water vapour) during the night. These can be abrupt and completely uncorrelated with airmass, which means we do not have a proxy for those changes (ask me more if you are interested). 

Additionally, there are also broader residual structures visible (broad stripes, sometimes with a checkerboard pattern). These are more instrumental in nature, and may depend on the overall change in response of the spectrograph, or in the temperature of the detector, etc. **These effects are quite common in the NIR and this is why techniques suitable for HRCCS in the optical do not work effectively here.**

**NOTE: telluric correction via e.g. MolecFit can achieve similar level of correction (1-2%) in the infrared as our simplest algorithm designed above.**

Typically, we apply a 2nd-order correction of telluric lines by *sampling* a couple of strong residuals (e.g. the lines close to 2322 nm and 2328.5 nm) and modelling the flux in every column via a multi-linear regression of these residuals. 

More advanced approaches skip the airmass detrending at all and try to find a set of vectors (eigenvectors) than can be combined linearly to describe the time variation of flux in each column. Two of these techinques are Principal Component Analysis and the weighted variant Sysrem (you'll hear about Sysrem from Neale Gibson).

In this context, however, I am mostly interested to show you what happens if we take the simplest and most basic route, which is masking the noisy channels. We won't probably achieve a result that is as good as in the paper, but it will still make our point.

In [ ]:
''' Designing a brute-force mask by looking at the standard dev along a column '''

spc = tRem[0,].copy()    # Copy of 1st CRIRES detector post airmass detrending
std = np.std(spc,axis=0) # Standard deviation along each column (time axis)

thre = 1.8 * np.median(std)  # Testing a threshold for masking

# Plotting the threshold to convince ourselves that it makes sense
plt.plot(wlen[0,],std)
plt.plot(wlen[0,],thre*np.ones(nPix))
plt.show()

Let's write the code that masks the noisiest spectral channels. Let's also weigh the remaining columns by their variance. Note that this was not necessary in the HARPS data because the wavelength-dependent flux variations were preserved, while here we normalise each wavelength.

In [ ]:
''' LIVE CODE - Designing a function that masks noisy columns and weighs unmasked by their variance '''

def mask_residuals(cube,thre=2.0):
    nD, nF, nP = cube.shape
    masked = cube.copy() - 1.0
    std = np.std(masked,axis=1)   # It has dimension (nDet, nPix)
    for iD in range(nD):
        mask1 = (std[iD,] > thre * np.median(std[iD,]))
        mask2 = (std[iD,] < 0.2*np.median(std[iD,]))
        mask = mask1 + mask2
        masked[iD,:,mask] = 0.0
        good = mask == False
        for iF in range(nF):
            masked[iD,iF,good] /= std[iD,good]**2
        masked[iD,] *= np.mean(std[iD,good]**2)
    return masked    
    
masked = mask_residuals(tRem, thre=1.8)
for iDet in range(nDet): plotMatrix(masked[iDet,],wlen[0,],ph,stretch=True)

## Part 3 - Cross correlating the data

We are now going to load the best-fitting model spectrum for the atmosphere of HD 179949 b (as per Brogi et al. 2014) and use it to cross correlate the telluric-removed data. 

In [ ]:
wMod, fMod = np.loadtxt('spec_hd179949.dat',unpack=True)
wMod *= 1E3
isort = np.argsort(wMod)
wMod = wMod[isort]
fMod = fMod[isort]
fMod /= np.max(fMod)

plt.figure(figsize=(15,3))
plt.plot(wMod, fMod)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Flux (normalised)')
plt.show()

### 3.1 - Spline versus linear interpolation
While Jens introduced the concept of interpolation, which is used to Doppler shift the model spectra, here I would like to use *spline* interpolation rather than *linear* interpolation. If you have a model at *R* much greater than the data *R*, it won't make a difference which interpolation you choose. Here, however, the model has an equivalent *R* of about 250,000, which is not very far from the pixel sampling of CRIRES (1.5 km / s / pixel - equivalent to *R* = 100,000 with two pixels per resolution element). If we use linear interpolation, we risk to make small errors every time the model is shifted. Here is an example:

In [ ]:
''' Selecting a small range '''
idx = (wMod < 2299.9) * (wMod>2299.7)
wl = wMod[idx]
fl = fMod[idx]
plt.plot(wl,fl,'x',label='input')

''' Requesting an oversampled output vector '''
wlOut = np.arange(2299.7,2299.9,0.001)

''' Computing the spline interpolation '''
cs = interpolate.splrep(wMod,fMod,s=0)
spline_int = interpolate.splev(wlOut,cs,der=0)

''' Computing the linear interpolation'''
linear_int = interpolate.interp1d(wMod,fMod)(wlOut)

plt.plot(wlOut,spline_int,label='spline')
plt.plot(wlOut,linear_int,label='linear')
plt.legend()
plt.show()

### 3.2 - Cross correlating the whole sequence
Let's now build the function that computes the CCF ona fixed RV grid. While this is analogous to what Jens just explained, we'll be modifying a couple of details. Firstly, we are going to re-write the cross correlation formula but using matrix multiplication, which is significantly faster in Python than the <code>.sum()</code> Numpy method.

For reference, remember that the cross correlation $C$ between two vectors $f$ and $g$ is
$$ C = \frac{\frac{1}{N}\sum_if_ig_i}{\frac{1}{N}\sqrt{\sum_if_i^2\sum_ig_i^2}}$$

In [ ]:
''' Live code - writing a faster xcorr code '''
def xcorr(f,g):
    nx = len(f)
    I = np.ones(nx)
    f -= np.dot(f,I)/nx
    g -= np.dot(g,I)/nx
    R = np.dot(f,g)/nx
    varf = np.dot(f,f)/nx
    varg = np.dot(g,g)/nx
    return R / np.sqrt(varf*varg)

''' Keeping this here - previous version '''
def xcorr_slow(f,g):
    nx = len(f)
    f -= np.mean(f)
    g -= np.mean(g)
    R = np.sum(f*g)/nx
    varf = np.sum(f*f)/nx
    varg = np.sum(g*g)/nx
    return R / np.sqrt(varf*varg)

Now we are going to build our lag vector and loop over all the detectors and all the spectra to build a cross correlation cube as a function of detector, time, and RV lag.

In [ ]:
''' Defining lag vector '''
ncc = 333               # Number of cross correlation values
dRV = 1.5
rvlag = (np.arange(ncc) - (ncc-1)//2) * dRV     # RV lag centred around zero and with step 1.5 km/s
print(rvlag[0],rvlag[-1])

''' Exploring the the CCF grid '''
def get_cc_grid_slow(wMod,fMod,wlen):
    ccf = np.zeros((nDet,nObs,ncc))
    for iDet in range(nDet):
        for irv, rv in enumerate(rvlag):
            beta = rv / 2.998E5
            wModShifted = wMod * np.sqrt( (1+beta)/(1-beta) )
            coef_spline = interpolate.splrep(wModShifted,fMod,s=0)
            intMod = interpolate.splev(wlen,coef_spline,der=0)
            for iObs in range(nObs):
                ccf[iDet,iObs,irv] = xcorr_slow(masked[iDet,iObs,], intMod)
    return ccf

Here is a faster version which tries to use the fact that spline interpolation is split in two steps, the computation of the spline through <code>interpolate.splrep()</code> and its evaluation on any output vector via <code>interpolate.splev()</code>. Rather than shifting the model wavelenghts and having to compute <code>ncc</code> times both functions, we can compute just once <code>.splrep()</code> using the unshifted model wavelength, and shift the output vector (the data wavelenghts) instead. This is a completely symmetric change of reference frame and can ve simply achieved by swapping the sign of RV *as long as you use the relativistic formula*.

In [ ]:
''' Example of faster version '''
def get_cc_grid(wMod,fMod,wlen):
    ccf = np.zeros((nDet,nObs,ncc))
    coef_spline = interpolate.splrep(wMod, fMod, s=0.0)
    for irv, rv in enumerate(rvlag):
        beta = rv / 2.998E5
        # Shifting data wlen instead by swapping sign of RV
        wShift = wlen * np.sqrt( (1-beta) / (1+beta) )  # A (nDet,nPix) vector
        intMod = interpolate.splev(wShift,coef_spline,der=0) # A (nDet,nPix) vector
        for iDet in range(nDet):
            for iObs in range(nObs):
                ccf[iDet,iObs,irv] = xcorr(masked[iDet,iObs,], intMod[iDet,])
    return ccf

import time as tim

t0 = tim.time()
ccf = get_cc_grid_slow(wMod,fMod,wlen)
print('Slow version: cross correlation cube computed in {:4.1f} seconds'.format(tim.time()-t0))

t0 = tim.time()
ccf = get_cc_grid(wMod,fMod,wlen)
print('Cross correlation cube computed in {:4.1f} seconds'.format(tim.time()-t0))

Lastly, we are going to sum the cross correlation over the 3 detectors used, and also compute the weights of each CCF (i.e. the CCF coming from each spectrum in the sequence) to be used at a later stage.

In [ ]:
ccfSum = ccf.sum(axis=0)
ccfWeight = np.sum(medFlux,axis=0)
ccfWeight /= ccfWeight.sum()
nObs = len(ccfWeight)
for iObs in range(nObs): ccfSum[iObs,] -= np.median(ccfSum[iObs,])
plotMatrix(ccfSum,rvlag,ph,'Radial velocity (km/s)','Orbital phase',stretch=True)

Compared to the Kelt-9 b transmission spectroscopy, here the planet signal isn't really visible, and this is due to the much smaller signal expected, on the order of 0.001x the stellar spectrum. 

To detect the planet's atmosphere, we need an additional step when we **coadd** all the CCFs in time (i.e. along the vertical axis). The problem is, these RVs are calculated in the observer's reference frame, and thus the planet will shift in RV across the sequence, i.e. its signal will look like a slanted line (remember the Kelt-9b final result). 

To properly co-add in time, for each phase we'll need to **pick the value of CC corresponding to the planet's rest frame.** Let's rewrite the planet RV equation in the observer's reference frame:
$$ RV_P = V_{sys} - V_{bary} + K_p \sin(2\pi\varphi) $$
$V_{sys}-V_{bary}$ is our <code>rvel</code> vector as explained above. $K_p$ is the maximum amplitude of the planet orbital radial velocity (i.e. the orbital velocity times $\sin(i)$) and $\varphi$ is the orbital phase.

Let me show you how different choices of $K_p$ or an overall offset would look like in the cross correlation matrix we just computed.

In [ ]:
plt.figure(figsize=(12,3))
plt.contourf(rvlag,ph,ccfSum,levels=256)
rv1 = rvel + 80*np.sin(2*np.pi*ph)
rv2 = rvel + 80*np.sin(2*np.pi*ph) + 20
rv3 = rvel + 142*np.sin(2*np.pi*ph)
plt.plot(rv1,ph,'w')
plt.plot(rv2,ph,'tab:orange')
plt.plot(rv3,ph,'y')
plt.show()

HD 179949 b is a non-transiting exoplanet: the orbital inclination is unknown and the zero-point in phase is uncertain. But even for transiting planets, planet RVs are only known to a few km/s precision, well above the resolution of these observations. Lastly, only co-adding at the planet RV does not reveal possible systematic noise that mimics the presence of the planet at different RVs. Therefore the common approach is to co-add the planet RV in time based on a grid of orbital parameters. 

We thus need to write a function that explores both a change in slope ($K_p$), and a global *horizontal shift* of the CCF($V_{rest}$). For each couple of velocities, we need to **shift the sequence of CCFs to the planet's rest frame** via interpolation and **co-add in time**. If we are smart, we can get the $V_{rest}$ for free through interpolation.

In [ ]:
# Defining vectors for co-adding of the CCFs
nKp = 81
nVr = 81
kpVec = np.arange(nKp) - (nKp-1)//2 + 145     # Centred on the injected RV
vrest = (np.arange(nVr) - (nVr-1)//2)*1.5     # Centred on zero rest-frame velocity

dphi = -0.0077   # From the 2014 paper

''' LIVE CODE here '''
def get_velocity_map(ccfSum,ccfW,rvel,ph,dphi):
    ccfTot = np.zeros((nKp,nVr))
    for ikp in range(nKp):
        rvPl = rvel + kpVec[ikp] * np.sin(2.0*np.pi*(ph+dphi))
        for iObs in range(nObs):
            outRV = vrest + rvPl[iObs]
            fit = interpolate.interp1d(rvlag, ccfSum[iObs,])
            ccfTot[ikp,] += fit(outRV)# * ccfWeight[iObs]
    return ccfTot

ccfTot = get_velocity_map(ccfSum,ccfWeight,rvel,ph,dphi)

I am nit going to teach you here how to measure the significance of the detection with statistical tests such as t-test. I'll just use S/N as a proxy.

In [ ]:
''' Converting to SNR - Dividing each CC value by the std of the values far from the peak. '''
ccfTot /= np.std(ccfTot[:,np.abs(vrest)>20])

In [ ]:
lims = [vrest[0],vrest[-1],kpVec[0],kpVec[-1]]
plt.imshow(ccfTot,origin='lower',extent=lims,aspect='auto')
plt.xlabel('Rest-frame velocity (km/s)')
plt.ylabel('Max. orbital RV (km/s)')
plt.colorbar()
plt.show()

# Asking the code to locate the peak
ipeak = np.where(ccfTot == ccfTot.max())
bestVr = float(vrest[ipeak[1]])
bestKp = float(kpVec[ipeak[0]])
print('Peak position in Vrest = {:3.1f} km/s'.format(bestVr))
print('Peak position in Kp = {:6.1f} km/s'.format(bestKp))
print('Max SNR = {:3.1f}'.format(ccfTot.max()))

We can already see the detection, but what if we add a second night? Let's write a flexible code that allows us to specify as many nights as possible and repeats the analysis above by stacking the relevant vectors and matrices along the time axis. Here is where our previous function definition is handy!

In [ ]:
nights = ['3']   # List of nights to process

# Preparing vectors to append phases, RVs, CCFs, and weights
phLong = ph.copy()
rvLong = rvel.copy()
ccfLong = ccf.copy()
wwLong = medFlux.copy()

# Looping over nights of data
for night in nights:
    dataFold = 'night'+night+'/'
    print('Adding night {0}'.format(night))
    fl0, wl0, air0, rv0, ph0 = get_night_vectors(dataFold)
    nDet, nObs, nPix = fl0.shape   # Necessary to update global variables used inside functions
    phLong = np.append(phLong,ph0)
    rvLong = np.append(rvLong,rv0)
    fNorm, medFlux0 = normalise_continuum(fl0.copy())
    tRem = detrend_tellurics(air0,fNorm,deg=2)
    masked = mask_residuals(tRem)
    ccCube = get_cc_grid(wMod,fMod,wl0)
    wwLong = np.append(wwLong,medFlux0,axis=1)
    ccfLong = np.append(ccfLong,ccCube,axis=1)

In [ ]:
ccfSumLong = ccfLong.sum(axis=0)
ccfWeight = np.sum(wwLong,axis=0)
ccfWeight /= ccfWeight.sum()
nObs = len(ccfWeight)
for iObs in range(nObs): ccfSumLong[iObs,] -= np.median(ccfSumLong[iObs,])

''' Plotting after reordering phases '''
isort = np.argsort(phLong)
matrix = ccfSumLong[isort,]
yvec = phLong[isort]
plotMatrix(matrix,rvlag,yvec,'Radial velocity (km/s)','Orbital phase',stretch=True)

Let's now repeat the shifting and coadding and look at our (Vrest, Kp) matrix again.

In [ ]:
ccfTot = get_velocity_map(ccfSumLong,ccfWeight,rvLong,phLong,dphi)
ccfTot /= np.std(ccfTot[:,np.abs(vrest)>20])

plt.imshow(ccfTot,origin='lower',extent=lims,aspect='auto')
plt.xlabel('Rest-frame velocity (km/s)')
plt.ylabel('Max. orbital RV (km/s)')
plt.colorbar()
plt.show()

# Asking the code to locate the peak
ipeak = np.where(ccfTot == ccfTot.max())
bestVr = float(vrest[ipeak[1]])
bestKp = float(kpVec[ipeak[0]])
print('Peak position in Vrest = {:3.1f} km/s'.format(bestVr))
print('Peak position in Kp = {:6.1f} km/s'.format(bestKp))
print('Max SNR = {:3.1f}'.format(ccfTot.max()))

It worked! We got a better signal (better localised and higher SNR) by adding spectra, which is great! In this example we are not going to add the second night because our simplified telluric removal does not work well in this case, but I'll include the data anyway for you to test:
<ul>
    <li>Change <code>nights = ['3']</code> to <code>nights = ['2','3']</code> and check that the detection goes down.
        <li>Repeat the whole analysis changing the folder at the top to <code>dataFold = night2/</code> and try to understand why the second night does not work.
</ul>
You now know how to look for the atmosphere of non-transiting planets with HRCCS!